In [1]:
embedding_name = 'w2v_hybrid'
num_threads = 4
train_name = 'train.csv'
train_name = 'test.csv'
mode = 'chinese' # english / chinese
vocab_name = 'vocab.json'
sent2seq_name = 'sent2seq.json'

In [2]:
from multiprocessing.pool import ThreadPool as Pool
import json
from tqdm import tqdm_notebook as tqdm
import jieba.posseg as pseg
import csv

In [3]:
BOS = '<bos>'
EOS = '<eos>'
PAD = '<pad>'
UNK = '<unk>'
MAX_Q_LEN = 20
MAX_A_LEN = 20

In [5]:
##### Data Loader
print("Loading {} ...".format(train_name))
sents = {}
Reader = csv.reader(open(train_name, newline='', encoding='utf-8'), delimiter=',', quotechar='"')
for i,fields in enumerate(Reader):    
    if i == 0:
        continue
    tid1, tid2 = fields[1:3]
    if mode == 'english':
        sent1 = fields[5]
        sent2 = fields[6]
    elif mode == 'chinese':
        sent1 = fields[3]
        sent2 = fields[4]
    if tid1 not in sents:
        sents[tid1] = sent1 
    if tid2 not in sents:
        sents[tid2] = sent2 
NUM_DATA = len(sents)
print("done. {} data loaded.".format(NUM_DATA))

Loading train.csv ...
done. 167564 data loaded.


In [6]:
##### Make lexicon (no need to re-run)

vocab = {'<pad>':0, '<bos>':1, '<eos>':2, '<unk>': 3}

for key, sent in tqdm(sents.items()):
    words = pseg.cut(sent)
    out_seq = []
    for w,flag in words:
        if flag is not 'x':
            try:
                wid = vocab[w]
            except KeyError:
                wid = len(vocab)
                vocab[w] = wid
            out_seq.append(wid)
    sents[key] = out_seq
    
print('dumping data to ' + vocab_name)
json.dump(vocab, open(vocab_name, 'w', encoding='utf-8'))
json.dump(sents, open(sent2seq_name, 'w'))
print('done')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.749 seconds.
Prefix dict has been built succesfully.



dumping data to vocab.json
done
